In [1]:
import requests
import json
import time
from tqdm import tqdm
from balance_date import BalanceDate

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

import utils

配置文件
{'aaindex': './data/aaindexmatrix.txt',
 'all_feature': './data/all_feature.csv',
 'feature_kind': './data/features_kind.xls',
 'feature_selected': {'LightGBM_rfe_30': './out/20210108/10cv_3class_feature_select_LightGBM_30.rfe',
                      'RandomForeast': './out/20201224/10cv_3class_feature_select_RandomForeast.rfecv',
                      'RandomForeast_rfe_30': './out/20201226/10cv_3class_feature_select_RandomForeast_30.rfe',
                      'XGBoost': './out/20201224/10cv_3class_feature_select_XGBoost.rfecv',
                      'XGBoost_rfe_30': './out/20201226/10cv_3class_feature_select_XGBoost_30.rfe',
                      'lightGBM': './out/20210108/10cv_3class_feature_select_LightGBM.rfecv',
                      'lightGBM_layer1_-1': './out/20210106/lightGBM_feature_select_for_-1_layer1.rfecv',
                      'lightGBM_layer1_-1_ref_30': './out/20210106/lightGBM_feature_select_for_-1_layer1_30_features.rfe',
                      'lightGBM_lay

# 获取结果函数

In [2]:
def submit(gi, sequence, change=None):
    """ 提交查询请求 """
    url = "https://protein-sol.manchester.ac.uk/cgi-bin/solubility/sequenceprediction.php"
    sequence = sequence.replace("X", "A")  # 有少数是X，但是不允许出现，这里改为A
    if change is not None:
        # 如果存在变异
        print("存在变异", change)
        fr, index, t = change[0], change[1: -1], change[-1]
        index = int(index) - 1
        if sequence[index] != fr:
            raise RuntimeError("序列不匹配")
        sequence = sequence[:index] + t + sequence[index+1:]
    payload = (
        ("sequence-input", ">gi{}\n{}".format(gi, sequence)), 
        ("singleprediction", "提交"),
    )
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        job_id = re.search("<p>Job id.*?=.*?(\w+)</p>", response.text, flags=re.I).group(1)
        print("获取到 job id：", job_id)
    else:
        raise RuntimeError("网页访问错误，状态码为 %s" % response.status_code )
    return job_id

In [3]:
def get_protein_sol_result(job_id):
    """ 根据 job id 获取结果"""
    url = "https://protein-sol.manchester.ac.uk/cgi-bin/utilities/download_file.php"
    payload = (
        ("app","solubility"),
        ("dirname","run"),
        ("timestamp", job_id),
        ("idname", job_id),
        ("file","seq_prediction.txt"),
    )
    response = requests.get(url,params=payload)

    if response.status_code == 200:
        search_solubility = re.search("SEQUENCE PREDICTIONS,(.*)", response.text).group(1)
        solubility = search_solubility.split(",")[2]
        solubility = float(solubility)
        print("获取到 solubility：", solubility)
    else:
        raise RuntimeError("网页访问错误，状态码为 %s" % response.status_code )
    return solubility

In [4]:
def get_solubility(gi, sequence, change=None):
    print("开始查询 gi：", gi)
    job_id = submit(gi, sequence, change)
    solubility = get_protein_sol_result(job_id)
    return solubility

In [5]:
#测试
gi = "P00547"
change = "M1A"
sequence = "MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSLNNLGRFADKLPSEPRENIVYQCWERFCQELGKQIPVAMTLEKNMPIGSGLGSSACSVVAALMAMNEHCGKPLNDTRLLALMGELEGRISGSIHYDNVAPCFLGGMQLMIEENDIISQQVPGFDEWLWVLAYPGIKVSTAEARAILPAQYRRQDCIAHGRHLAGFIHACYSRQPELAAKLMKDVIAEPYRERLLPGFRQARQAVAEIGAVASGISGSGPTLFALCDKPETAQRVADWLGKNYLQNQEGFVHICRLDTAGARVLEN"
get_solubility(gi, sequence, change)

开始查询 gi： P00547
存在变异 M1A
获取到 job id： 91595d8ddec7b9f28cf5
获取到 solubility： 0.336


0.336

# 读取数据

In [6]:
# 定义常量
OUT_PATH = os.path.join(".", "out", time.strftime("%Y%m%d"))  # 输出路径
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
print("输出路径为：", OUT_PATH)

输出路径为： ./out/20210116


In [7]:
config_path = "./config.json"
with open(config_path) as f:
    config = json.loads(f.read())
config

{'origin_data': './data/PON-Sol_data - YSD=0.15.xlsx',
 'yeast_lgk': './data/yeast_lgk.csv',
 'yeast_tem': './data/yeast_tem.csv',
 'genes_seq': './data/all_genes.csv',
 'aaindex': './data/aaindexmatrix.txt',
 'sift_result': './data/sift_data/result1_90_100/sift_result',
 'sift_aligedfasta': './data/sift_data/result1_90_100',
 'all_feature': './data/all_feature.csv',
 'feature_kind': './data/features_kind.xls',
 'feature_selected_dir': './data/result/feature selected/',
 'feature_selected': {'RandomForeast': './out/20201224/10cv_3class_feature_select_RandomForeast.rfecv',
  'XGBoost': './out/20201224/10cv_3class_feature_select_XGBoost.rfecv',
  'lightGBM': './out/20210108/10cv_3class_feature_select_LightGBM.rfecv',
  'lightGBM_layer1_-1': './out/20210106/lightGBM_feature_select_for_-1_layer1.rfecv',
  'lightGBM_layer2_-1': './out/20210106/lightGBM_feature_select_for_-1_layer2.rfecv',
  'RandomForeast_rfe_30': './out/20201226/10cv_3class_feature_select_RandomForeast_30.rfe',
  'LightGBM

In [8]:
# 获取基因序列文件
genes = pd.read_csv(config["genes_seq"])
genes

,num,gi,name,seq
0,0,109731493,>AAI14500.1 Tryptophan hydroxylase 2 [Homo sap...,MQPAMMMFSSKYWARRGFSLDSAVPEEHQLLGSSTLNKPNSGKNDD...
1,1,11133565,>sp|Q9NPJ1.1|MKKS_HUMAN RecName: Full=McKusick...,MSRLEAKKPSLCKSEPLTTERVRTTLSVLKRIVTSCYGPSGRLKQL...
2,2,116242717,>sp|Q13563.3|PKD2_HUMAN RecName: Full=Polycyst...,MVNSSRVQPQQPGDAKRPPAPRAPDPGRLMAGCAAVGASLAAPGGL...
3,3,116666949,">pdb|2CY8|A Chain A, D-phenylglycine Aminotran...",MSILNDYKRKTEGSVFWAQRARSVMPDGVTADTRAFDPHGLFISDA...
4,4,117391,>sp|P12271.2|RLBP1_HUMAN RecName: Full=Retinal...,MSEGVGTFRMVPEEEQELRAQLEQLTTKDHGPVFGPCSQLPRHTLQ...
...,...,...,...,...
72,72,82407494,">pdb|1XWW|A Chain A, Low Molecular Weight Phos...",AEQATKSVLFVCLGNICRSPIAEAVFRKLVTDQNISENWVIDSGAV...
73,73,8248030,>AAF74023.1 aryl-hydrocarbon interacting prote...,MDAALLLNVEGVKKTILHGGTGELPNFITGSRVIFHFRTMKCDEER...
74,74,82581658,>sp|P0C0P0.1|DYR_STAEP RecName: Full=Dihydrofo...,MTLSIIVAHDKQRVIGYQNQLPWHLPNDLKHVKQLTTGNTLVMGRK...
75,75,83754916,">pdb|2D26|A Chain A, Active Site Distortion Is...",MDPQGDAAQKTDTSHHDQDHPTFNKITPNLAEFAFSLYRQLAHQSN...


In [9]:
# 获取盲测集
X_test1, y_test1, X_test2, y_test2 = BalanceDate.get_test()  # 两组测试数据  
X_test1 = pd.merge(X_test1, genes, left_on="gi", right_on="gi").loc[:, ["index", "gi", "solubility", "variation", "mut_residue", "mut_from", "mut_to", "seq"]]
X_test2 = pd.merge(X_test2, genes, left_on="gi", right_on="gi").loc[:, ["index", "gi", "solubility", "variation", "mut_residue", "mut_from", "mut_to", "seq"]]

# 训练

In [11]:
# 所有的蛋白质进行判断solubility
genes["protein-sol"] = genes.apply(lambda x: get_solubility(x.gi, x.seq), axis=1)

# 获取所有测试集当前变异所对应的solubility
X_test1["protein-sol"] = X_test1.apply(lambda x: get_solubility(x.gi, x.seq, x.variation), axis=1)
X_test2["protein-sol"] = X_test2.apply(lambda x: get_solubility(x.gi, x.seq, x.variation), axis=1)

开始查询 gi： 109731493
获取到 job id： 205617db4d7a66553447
获取到 solubility： 0.406
开始查询 gi： 11133565
获取到 job id： 1a8dd317a7997a93c693
获取到 solubility： 0.314
开始查询 gi： 116242717
获取到 job id： 487472956bd2ebc41b36
获取到 solubility： 0.298
开始查询 gi： 116666949
获取到 job id： 53629caf7741aa8ff949
获取到 solubility： 0.227
开始查询 gi： 117391
获取到 job id： 7a0183f9d45d3a811354
获取到 solubility： 0.516
开始查询 gi： 117544
获取到 job id： 39ca473fe61791e8b5cf
获取到 solubility： 0.405
开始查询 gi： 118137700
获取到 job id： a2c7a5393d249649895f
获取到 solubility： 0.718
开始查询 gi： 121069
获取到 job id： f5de23530d8fa81923ff
获取到 solubility： 0.318
开始查询 gi： 1236048426
获取到 job id： 769ea01b38c8a56d8c8d
获取到 solubility： 0.48
开始查询 gi： 123845644
获取到 job id： 62f55f72657b447ea8b1
获取到 solubility： 0.616
开始查询 gi： 124831
获取到 job id： 8ce26c7af432dbe8f35c
获取到 solubility： 0.546
开始查询 gi： 127455
获取到 job id： 97d156ac7ce35dc42cdb
获取到 solubility： 0.389
开始查询 gi： 133854437
获取到 job id： 9b2a41125e03ab7d0d6d
获取到 solubility： 0.463
开始查询 gi： 1351170
获取到 job id： d69c28c9d8f4bfaf8948
获取到 

获取到 job id： 7f8f471380bc3a3059e4
获取到 solubility： 0.509
开始查询 gi： 326936885
存在变异 V126K
获取到 job id： a95642d4cd0fdfa66bb2
获取到 solubility： 0.509
开始查询 gi： 326936885
存在变异 V165K
获取到 job id： 335cbe90cf06d68f8ed5
获取到 solubility： 0.509
开始查询 gi： 326936885
存在变异 V176K
获取到 job id： 6f7006e6b5c68c6b787d
获取到 solubility： 0.509
开始查询 gi： 326936885
存在变异 V79K
获取到 job id： 8955a167867c59f3173d
获取到 solubility： 0.509
开始查询 gi： 326936885
存在变异 W132K
获取到 job id： 65bed62b31255d100f1f
获取到 solubility： 0.517
开始查询 gi： 326936885
存在变异 Y143K
获取到 job id： 4307be5d7b2ac7256017
获取到 solubility： 0.516
开始查询 gi： 326936885
存在变异 V113K
获取到 job id： 85be5a4fe0c66fa4d8bb
获取到 solubility： 0.509
开始查询 gi： 326936885
存在变异 I135K
获取到 job id： 2922d80bd70d7cbd0ae2
获取到 solubility： 0.509
开始查询 gi： 82581658
存在变异 N131D
获取到 job id： d8f2accf1d70bac6dd40
获取到 solubility： 0.544
开始查询 gi： 82581658
存在变异 N49E
获取到 job id： 802f3fce09dd8e378d57
获取到 solubility： 0.543
开始查询 gi： 82581658
存在变异 Q18D
获取到 job id： 1e7481617741e427fdc0
获取到 solubility： 0.545
开始查询 gi： 8258165

获取到 job id： 1b39d1a01ee8fb98419d
获取到 solubility： 0.421
开始查询 gi： 190610225
存在变异 R66H
获取到 job id： 72517c1976436cfdd892
获取到 solubility： 0.435
开始查询 gi： 190610225
存在变异 R66I
获取到 job id： b003802e0e98b9537f54
获取到 solubility： 0.429
开始查询 gi： 190610225
存在变异 R66K
获取到 job id： 37cecc28b05379f24b06
获取到 solubility： 0.43
开始查询 gi： 190610225
存在变异 R66N
获取到 job id： e9feced36a8ef12995b4
获取到 solubility： 0.437
开始查询 gi： 190610225
存在变异 R66P
获取到 job id： de9f2e7f53af6c1cea11
获取到 solubility： 0.434
开始查询 gi： 190610225
存在变异 R66Q
获取到 job id： 14898a3068e646224a32
获取到 solubility： 0.437
开始查询 gi： 190610225
存在变异 R66S
获取到 job id： d1aa8c7897dda3017458
获取到 solubility： 0.434
开始查询 gi： 190610225
存在变异 R66V
获取到 job id： 5620fe209e29ec52ee6d
获取到 solubility： 0.441
开始查询 gi： 190610225
存在变异 R66W
获取到 job id： 4175d28da08e6c3d2a9f
获取到 solubility： 0.425
开始查询 gi： 190610225
存在变异 I68A
获取到 job id： 5e50b857262755708903
获取到 solubility： 0.418
开始查询 gi： 190610225
存在变异 I68D
获取到 job id： 4aea943aa8ee2e8c1f85
获取到 solubility： 0.447
开始查询 gi： 190610225
存在变

获取到 job id： 466b32df849cd923533e
获取到 solubility： 0.434
开始查询 gi： 190610225
存在变异 R143V
获取到 job id： 2bfb9c5318aca721954d
获取到 solubility： 0.441
开始查询 gi： 190610225
存在变异 R143Y
获取到 job id： ed37261a3762fbeef77e
获取到 solubility： 0.425
开始查询 gi： 190610225
存在变异 I146A
获取到 job id： f95217f02572178166c9
获取到 solubility： 0.418
开始查询 gi： 190610225
存在变异 I146C
获取到 job id： 8536812abef488a21c5a
获取到 solubility： 0.418
开始查询 gi： 190610225
存在变异 I146E
获取到 job id： 2bc52c146520a085ea02
获取到 solubility： 0.447
开始查询 gi： 190610225
存在变异 I146G
获取到 job id： da518e61630c2ece199e
获取到 solubility： 0.42
开始查询 gi： 190610225
存在变异 I146K
获取到 job id： 21da14a90f5fbb74b469
获取到 solubility： 0.42
开始查询 gi： 190610225
存在变异 I146P
获取到 job id： 42280fc4119bf8afafc4
获取到 solubility： 0.421
开始查询 gi： 190610225
存在变异 I146Q
获取到 job id： bf7989e85ffe9daa00fc
获取到 solubility： 0.423
开始查询 gi： 190610225
存在变异 I146R
获取到 job id： 6f6309081870dea138d7
获取到 solubility： 0.405
开始查询 gi： 190610225
存在变异 I146S
获取到 job id： 368f1c58a4697f2f2753
获取到 solubility： 0.42
开始查询 gi： 1906

获取到 job id： 04d7ad392b503d0e2477
获取到 solubility： 0.416
开始查询 gi： 190610225
存在变异 I307N
获取到 job id： 3b8c2c98d3232bd5a9bd
获取到 solubility： 0.423
开始查询 gi： 190610225
存在变异 I307P
获取到 job id： 738889be6bb8e1fe0388
获取到 solubility： 0.421
开始查询 gi： 190610225
存在变异 I307R
获取到 job id： 7526d7d1db80044c1733
获取到 solubility： 0.405
开始查询 gi： 190610225
存在变异 I307S
获取到 job id： 503950adfea208061689
获取到 solubility： 0.42
开始查询 gi： 190610225
存在变异 I307V
获取到 job id： 536d88d399ba214c6264
获取到 solubility： 0.427
开始查询 gi： 190610225
存在变异 I307Y
获取到 job id： e34366c65cfacbde4e76
获取到 solubility： 0.411
开始查询 gi： 190610225
存在变异 N334C
获取到 job id： b6c9acdd97e39749ee69
获取到 solubility： 0.41
开始查询 gi： 190610225
存在变异 N334G
获取到 job id： 2f52e393273f4d30a25c
获取到 solubility： 0.413
开始查询 gi： 190610225
存在变异 N334H
获取到 job id： ddef544b16f836ec26de
获取到 solubility： 0.413
开始查询 gi： 190610225
存在变异 N334K
获取到 job id： 59089a65a1995a527bcc
获取到 solubility： 0.412
开始查询 gi： 190610225
存在变异 N334P
获取到 job id： a2b0fa20362b2bd78ecd
获取到 solubility： 0.414
开始查询 gi： 190

获取到 job id： 10ba47a2fb0360ddd05c
获取到 solubility： 0.301
开始查询 gi： 213133708
存在变异 L79T
获取到 job id： d7b658e02f061e79b5e0
获取到 solubility： 0.318
开始查询 gi： 213133708
存在变异 L79V
获取到 job id： 520a247cc127e6646220
获取到 solubility： 0.316
开始查询 gi： 213133708
存在变异 L79W
获取到 job id： 8129c9772a7e8702dd3c
获取到 solubility： 0.301
开始查询 gi： 213133708
存在变异 L79Y
获取到 job id： cb04e7a3229195b480c1
获取到 solubility： 0.301
开始查询 gi： 213133708
存在变异 Q86A
获取到 job id： faeb5f054973caff3de1
获取到 solubility： 0.328
开始查询 gi： 213133708
存在变异 Q86D
获取到 job id： f847e5b76be07382683e
获取到 solubility： 0.351
开始查询 gi： 213133708
存在变异 Q86E
获取到 job id： 5d518c75128c1592f9e6
获取到 solubility： 0.352
开始查询 gi： 213133708
存在变异 Q86F
获取到 job id： 89dc3101fe1d666c1fc8
获取到 solubility： 0.314
开始查询 gi： 213133708
存在变异 Q86G
获取到 job id： a4815f1e999cfc2af296
获取到 solubility： 0.327
开始查询 gi： 213133708
存在变异 Q86I
获取到 job id： cfa3e5223d3ebda6cadd
获取到 solubility： 0.326
开始查询 gi： 213133708
存在变异 Q86K
获取到 job id： 553a6213e218c0d601be
获取到 solubility： 0.314
开始查询 gi： 213133708
存在

获取到 job id： 436ebb86e4423062d6d7
获取到 solubility： 0.31
开始查询 gi： 213133708
存在变异 I140D
获取到 job id： 5160092d30b30e5b437d
获取到 solubility： 0.347
开始查询 gi： 213133708
存在变异 I140E
获取到 job id： ff3f3ea9508b81834a10
获取到 solubility： 0.348
开始查询 gi： 213133708
存在变异 I140G
获取到 job id： d30c6fd08a68bd5dec42
获取到 solubility： 0.322
开始查询 gi： 213133708
存在变异 I140H
获取到 job id： 22431e0e45bf9099aed0
获取到 solubility： 0.288
开始查询 gi： 213133708
存在变异 I140K
获取到 job id： 4b7b20b60b84a884317b
获取到 solubility： 0.309
开始查询 gi： 213133708
存在变异 I140L
获取到 job id： 14578527e9f3f01a4498
获取到 solubility： 0.326
开始查询 gi： 213133708
存在变异 I140N
获取到 job id： da00eda7e701397d6ce7
获取到 solubility： 0.317
开始查询 gi： 213133708
存在变异 I140P
获取到 job id： 7778dd44191a5737d9d6
获取到 solubility： 0.319
开始查询 gi： 213133708
存在变异 I140Q
获取到 job id： 59a7f932bac01c8abcc1
获取到 solubility： 0.317
开始查询 gi： 213133708
存在变异 I140R
获取到 job id： 671ff5d6ba42b9e11288
获取到 solubility： 0.306
开始查询 gi： 213133708
存在变异 I140S
获取到 job id： 8def08c1db11b89c7271
获取到 solubility： 0.321
开始查询 gi： 21

获取到 job id： 600a94a2f0af9db1eb13
获取到 solubility： 0.338
开始查询 gi： 213133708
存在变异 R176S
获取到 job id： 1e9c741b6421a153ae61
获取到 solubility： 0.338
开始查询 gi： 213133708
存在变异 R176T
获取到 job id： 68ae94b301e635b887a3
获取到 solubility： 0.34
开始查询 gi： 213133708
存在变异 R176V
获取到 job id： 1af4241737ee74bafa33
获取到 solubility： 0.338
开始查询 gi： 213133708
存在变异 R176W
获取到 job id： 33303912203469e67054
获取到 solubility： 0.322
开始查询 gi： 213133708
存在变异 R176Y
获取到 job id： 196b23ad046b3a92af4d
获取到 solubility： 0.322
开始查询 gi： 213133708
存在变异 A185G
获取到 job id： b0f8794b3d31caedc266
获取到 solubility： 0.319
开始查询 gi： 213133708
存在变异 A185N
获取到 job id： 6452bca584d4dda14a0d
获取到 solubility： 0.313
开始查询 gi： 213133708
存在变异 T186A
获取到 job id： b884d5f8d531f9ed77d4
获取到 solubility： 0.323
开始查询 gi： 213133708
存在变异 T186E
获取到 job id： ea5442518042d317906f
获取到 solubility： 0.347
开始查询 gi： 213133708
存在变异 T186G
获取到 job id： e62e86b8c1702e1c13b0
获取到 solubility： 0.322
开始查询 gi： 213133708
存在变异 T186H
获取到 job id： 650a896e66e62d12364e
获取到 solubility： 0.287
开始查询 gi： 21

In [12]:
_test1_path = os.path.join(OUT_PATH, "protein_sol_test1_result.xlsx")
_test2_path = os.path.join(OUT_PATH, "protein_sol_test2_result.xlsx")
X_test1.to_excel(_test1_path)
X_test2.to_excel(_test2_path)

print("输出路径为：")
print(_test1_path)
print(_test2_path)

输出路径为：
./out/20210116/protein_sol_test1_result.xlsx
./out/20210116/protein_sol_test2_result.xlsx
